In [ ]:
import API.Queries.Public.Futures as public_api
import Utils.DataModels as storage
import threading
import asyncio
import nest_asyncio
import Processor.ReciverData.DataHandler as handler

nest_asyncio.apply()

ins_public_api = public_api.API()

api_storage = storage.SymbolStorage()

symbols = ["BTCUSDT","TRXUSDT","ETHUSDT","XRPUSDT","SOLUSDT","BNBUSDT"]
intervals = ["1m", "3m", "5m", "15m"]#, "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M",]


In [15]:
for symbol in symbols:
    for interval in intervals:
        kline_data = ins_public_api.fetch_klines_limit(symbol=symbol, interval=interval, limit=1000)
        api_storage.update_data(symbol, interval, kline_data)
# print(api_storage)

In [ ]:
import concurrent.futures
import API.Queries.Public.Futures as public_api
import Utils.DataModels as storage
import threading

ins_public_api = public_api.API()

api_storage = storage.SymbolStorage()

symbols = ["BTCUSDT","TRXUSDT","ETHUSDT","XRPUSDT","SOLUSDT","BNBUSDT"]
intervals = ["1m", "3m", "5m", "15m"]#, "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M",]

def fetch_and_update(symbol, interval, api, storage):
    """심볼과 인터벌 데이터를 API에서 가져와 저장하는 함수"""
    kline_data = api.fetch_klines_limit(symbol=symbol, interval=interval, limit=1000)
    storage.update_data(symbol, interval, kline_data)

def threaded_data_fetch(symbols, intervals, api, storage):#, max_workers=100):
    """스레드를 사용하여 심볼과 인터벌별 데이터를 병렬로 가져옴"""
    max_workers = 5
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for symbol in symbols:
            for interval in intervals:
                futures.append(executor.submit(fetch_and_update, symbol, interval, api, storage))
        
        # 모든 작업 완료 대기
        concurrent.futures.wait(futures)

# 실행 예제
threaded_data_fetch(symbols, intervals, ins_public_api, api_storage)
# print(api_storage)

In [26]:
ins_public_api.fetch_klines_limit(symbol='BTCUSDT', interval='5m', limit=1)

[[1739623200000,
  '97695.80',
  '97700.00',
  '97616.50',
  '97688.70',
  '231.921',
  1739623499999,
  '22651831.81300',
  5161,
  '130.951',
  '12790778.66540',
  '0']]

In [31]:
import os

# CPU 코어 개수
cpu_cores = os.cpu_count()

print(f"사용 가능한 CPU 쓰레드 개수: {cpu_cores}")

사용 가능한 CPU 쓰레드 개수: 8


In [14]:
import datetime

def dummy(interval):
    current_time = datetime.datetime.now()    
    if "m" in interval:
        minutes = int(interval[:-1])
        if current_time.minute % minutes == 0:
            return True
    elif "h" in interval:
        hours = int(interval[:-1])
        if current_time.hour % hours == 0:
            return True
    else:
        hours = 24
        if current_time.hour % hours == 0:
            return True
    return False

In [17]:
dummy('6m')

False

In [18]:
a = '3m'
a[:-1]

'3'

In [27]:
import datetime

def is_time_match(interval):
    now = datetime.datetime.now()

    if interval.endswith("m"):  # 분 단위 확인
        return now.minute % int(interval[:-1]) == 0
    elif interval.endswith("h"):  # 시간 단위 확인
        return now.hour % int(interval[:-1]) == 0
    elif now.hour == 0 and now.minute == 0 and now.second == 0:  # 자정 확인
        return True

    return False

In [28]:
is_time_match('3m')

False